In [ ]:
pip install tweepy

pip install matplotlib

In [11]:
#initial imports
import tweepy 
import pandas as pd
import numpy as np 
from tweepy import OAuthHandler 
from tweepy import Cursor 
import nltk as nltk
import seaborn as sns
import re
import string
import warnings
warnings.filterwarnings('ignore')
nltk.download('vader_lexicon')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sarva\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
#Settings Twitter API Key
api_key='2LEkmltQnjgt4NEwAiU3wKfiU'
api_key_secret='Jg0Fl7wkEAVSbJnXpHXXZEySZka9Dg3FxW8H3tqCfaoFjrsowc'

acc_token='143141229-1gfi2d61Aco7beRwqgxTo8DdzjWmaKLmqPZgVepA'
acc_secret='4Cxbo1eNgS6DF69gF7vJ01IXE2hhU7Uo3qAjgvskhTTfo'

In [23]:
def get_twitter_auth():
    """
    @return:
        - the authentification to Twitter
    """
    try:
        consumer_key = api_key
        consumer_secret = api_key_secret
        access_token = acc_token
        access_secret = acc_secret
        
    except KeyError:
        sys.stderr.write("Twitter Environment Variable not Set\n")
        sys.exit(1)
        
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    return auth

def get_twitter_client():
    """
    @return:
        - the client to access the authentification API
    """
    auth = get_twitter_auth()
    client = tweepy.API(auth, wait_on_rate_limit=True)
    return client

def get_tweets_from_user(twitter_user_name, page_limit=16, count_tweet=20):
    """
    @params:
        - twitter_user_name: the twitter username of a user (company, etc.)
        - page_limit: the total number of pages (max=16)
        - count_tweet: maximum number to be retrieved from a page
        
    @return
        - all the tweets from the user twitter_user_name
    """
    client = get_twitter_client()
    
    all_tweets = []
    
    for page in Cursor(client.user_timeline, 
                        screen_name=twitter_user_name, 
                        count=count_tweet).pages(page_limit):
        for tweet in page:
            parsed_tweet = {}
            parsed_tweet['date'] = tweet.created_at
            parsed_tweet['author'] = tweet.user.name
            parsed_tweet['twitter_name'] = tweet.user.screen_name
            parsed_tweet['text'] = tweet.text
            parsed_tweet['number_of_likes'] = tweet.favorite_count
            parsed_tweet['number_of_retweets'] = tweet.retweet_count
                
            all_tweets.append(parsed_tweet)
    
    # Create dataframe 
    df = pd.DataFrame(all_tweets)
    
    # Revome duplicates if there are any
    df = df.drop_duplicates( "text" , keep='first')
    
    return df

#function for remove pattern in the input text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt


# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    #print(f"Compound: {score}, Result: {result}")
    
    return result



In [9]:
# making a dataframe with the bloomberg news
df1 = get_tweets_from_user("markets")
print("Data Shape: {}".format(df1.shape))
df1.columns = df1.columns.str.replace('text', 'tweet')
bloomberg = df1['tweet']
df1

# making a dataframe with the wall street journal news
df2 = get_tweets_from_user("wsj")
print("Data Shape: {}".format(df2.shape))
df2.columns = df2.columns.str.replace('text', 'tweet')
wall_street = df2['tweet']
df2

# making a dataframe with the yahoo finance news
df3 = get_tweets_from_user("yfinanceplus")
print("Data Shape: {}".format(df3.shape))
df3.columns = df3.columns.str.replace('text', 'tweet')
yahoo_finance = df3['tweet']
df3

#Concat all the tweets from the news accounts
a = {'Bloomberg':bloomberg.values, 'Wall Street':wall_street.values, 'Yahoo Finance':yahoo_finance}
all_tweet = pd.DataFrame.from_dict(a, orient='index')
all_tweet = all_tweet.transpose()
df = all_tweet
df["tweet"] = df["Bloomberg"].astype(str) + df["Wall Street"].astype(str) + df["Yahoo Finance"].astype(str)

Data Shape: (320, 6)
Data Shape: (320, 6)
Data Shape: (320, 6)


In [10]:
#remove twitter users
df['clean_tweet'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*")

#remove special characters, numbers and punctuations
df['clean_tweet'] = df['clean_tweet'].str.replace("[^a-zA-Z#]", " ")

#remove short words
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))



In [13]:
tokenized_tweet = df['clean_tweet'].apply(lambda x: x.split())
tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
#combine words into single sentence
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
df['clean_tweet'] = tokenized_tweet
df = df.drop(['tweet'], axis=1)


In [24]:
# Sentiment scores dictionaries
tweet_sent = {
    "tweet_compound": [],
    "tweet_pos": [],
    "tweet_neu": [],
    "tweet_neg": [],
    "tweet_sent": [],
}

# Get sentiment for the text and the title
for index, row in df.iterrows():
    try:
        # Sentiment scoring with VADER
        tweet_sentiment = analyzer.polarity_scores(row["clean_tweet"])
        tweet_sent["tweet_compound"].append(tweet_sentiment["compound"])
        tweet_sent["tweet_pos"].append(tweet_sentiment["pos"])
        tweet_sent["tweet_neu"].append(tweet_sentiment["neu"])
        tweet_sent["tweet_neg"].append(tweet_sentiment["neg"])
        tweet_sent["tweet_sent"].append(get_sentiment(tweet_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
tweet_sentiment_df = pd.DataFrame(tweet_sent)

# Describe dataframe
tweet_sentiment_df.describe()


,tweet_compound,tweet_pos,tweet_neu,tweet_neg,tweet_sent
count,320.000000,320.000000,320.000000,320.000000,320.000000
mean,0.112492,0.082306,0.855747,0.061950,0.181250
std,0.474812,0.063248,0.079979,0.059639,0.929299
min,-0.936000,0.000000,0.629000,0.000000,-1.000000
25%,-0.273200,0.044000,0.800750,0.000000,-1.000000
50%,0.140550,0.072000,0.861000,0.051000,1.000000
75%,0.526700,0.128250,0.916000,0.099250,1.000000
max,0.898000,0.287000,1.000000,0.294000,1.000000


In [25]:
# making a dataframe with the bloomberg news
df4 = get_tweets_from_user("CMTAssociation")
print("Data Shape: {}".format(df4.shape))
df4.columns = df4.columns.str.replace('text', 'tweet')
CMTAssociation = df4['tweet']
df4

# making a dataframe with the wall street journal news
df5 = get_tweets_from_user("ParetsJc")
print("Data Shape: {}".format(df5.shape))
df5.columns = df5.columns.str.replace('text', 'tweet')
ParetsJc = df5['tweet']
df5

# making a dataframe with the wall street journal news
df6 = get_tweets_from_user("allstarcharts")
print("Data Shape: {}".format(df6.shape))
df6.columns = df6.columns.str.replace('text', 'tweet')
allstarcharts = df6['tweet']
df6

#Concat all the tweets from the news accounts
a = {'CMT Association':CMTAssociation.values, 'RSI Wizard':ParetsJc.values, 'J.C. Parets':allstarcharts.values}
all_people_tweet = pd.DataFrame.from_dict(a, orient='index')
all_people_tweet = all_people_tweet.transpose()
new_df = all_people_tweet

#combine all tweets per news in one column
new_df["tweet"] = new_df["CMT Association"].astype(str) + new_df["RSI Wizard"].astype(str) + new_df["J.C. Parets"].astype(str)


Data Shape: (317, 6)
Data Shape: (34, 6)
Data Shape: (320, 6)


In [26]:
#remove twitter users
new_df['clean_tweet'] = np.vectorize(remove_pattern)(new_df['tweet'], "@[\w]*")

#remove special characters, numbers and punctuations
new_df['clean_tweet'] = new_df['clean_tweet'].str.replace("[^a-zA-Z#]", " ")

#remove short words
new_df['clean_tweet'] = new_df['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))



In [27]:
tokenized = new_df['clean_tweet'].apply(lambda x: x.split())
tokenized = tokenized.apply(lambda sentence: [stemmer.stem(word) for word in sentence])

#combine words into single sentence
for i in range(len(tokenized)):
    tokenized[i] = " ".join(tokenized[i])

new_df['clean_tweet'] = tokenized
new_df = new_df.drop(['tweet'], axis=1)



In [28]:
# Sentiment scores dictionaries
tweet_sent2 = {
    "tweet_compound2": [],
    "tweet_pos2": [],
    "tweet_neu2": [],
    "tweet_neg2": [],
    "tweet_sent2": [],
}

# Get sentiment for the text and the title
for index, row in new_df.iterrows():
    try:
        # Sentiment scoring with VADER
        tweet_sentiment2 = analyzer.polarity_scores(row["clean_tweet"])
        tweet_sent2["tweet_compound2"].append(tweet_sentiment2["compound"])
        tweet_sent2["tweet_pos2"].append(tweet_sentiment2["pos"])
        tweet_sent2["tweet_neu2"].append(tweet_sentiment2["neu"])
        tweet_sent2["tweet_neg2"].append(tweet_sentiment2["neg"])
        tweet_sent2["tweet_sent2"].append(get_sentiment(tweet_sentiment2["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
tweet_sentiment_df2 = pd.DataFrame(tweet_sent2)

# Describe dataframe
tweet_sentiment_df2.describe()

,tweet_compound2,tweet_pos2,tweet_neu2,tweet_neg2,tweet_sent2
count,320.000000,320.000000,320.000000,320.000000,320.000000
mean,0.222898,0.124269,0.832334,0.043400,0.393750
std,0.391021,0.127048,0.138622,0.079622,0.747802
min,-0.765000,0.000000,0.455000,0.000000,-1.000000
25%,0.000000,0.000000,0.743500,0.000000,0.000000
50%,0.273200,0.110500,0.843000,0.000000,1.000000
75%,0.526700,0.205000,1.000000,0.081250,1.000000
max,0.971700,0.545000,1.000000,0.459000,1.000000
